# Generating ESM embeddings from PDB files
  This is a colab for [generating ESM2 embeddings](https://github.com/facebookresearch/esm/tree/main) from sequences in PDB files.


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:

pip install biopython torch_geometric torch fair-esm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 661.6/661.6 kB 14.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 10.5 MB/s eta 0:00:00
  Created wheel for torch_geometric: filename=torch_geometric-2.3.1-py3-none-any.whl size=910454 sha256=05e026cf727a371b7fc9d37a6deec614449b2f6107328a87e9e2d7dc2b4617f7
  Stored in directory: /root/.cache/pip/wheels/ac/dc/30/e2874821ff308ee67dcd7a66dbde912411e19e35a1addda028
Successfully built torch_geometric


In [4]:
import os
import os.path as osp
import warnings
from math import pi as PI
from typing import Callable, Dict, Optional, Tuple

import numpy as np
import torch
import torch.nn.functional as F
from torch import Tensor
from torch.nn import Embedding, Linear, ModuleList, Sequential

from torch_geometric.data import Dataset, download_url, extract_zip
from torch_geometric.data.makedirs import makedirs
from torch_geometric.nn import MessagePassing, SumAggregation, radius_graph
from torch_geometric.nn.resolver import aggregation_resolver as aggr_resolver
from torch_geometric.typing import OptTensor

from Bio.PDB import PDBParser
from Bio import SeqIO

import esm

In [5]:
#%%bash
#rm -R embeddings
#rm -R fasta

### Setting data directories
We load PDB files from Google Drive and specify the output FASTA and embedding directories. Since the FASTA files aren't utilized downstream, we only save the embedding files to Drive.

In [6]:
label_file = '/content/drive/My Drive/RBG coding assignment/labels.txt'
data_dir = '/content/drive/My Drive/RBG coding assignment/data'

embeddings_dir = '/content/drive/MyDrive/esm_embed'
os.makedirs(embeddings_dir, exist_ok=True)
fasta_dir = 'fasta'
os.makedirs(fasta_dir, exist_ok=True)

In [7]:
files = os.listdir(data_dir)
files

['1a0g_1_protein.pdb',
 '1a05_1_protein.pdb',
 '1a0f_1_protein.pdb',
 '1a0j_1_protein.pdb',
 '1a0q_1_protein.pdb',
 '1a0t_1_protein.pdb',
 '1a1b_1_protein.pdb',
 '1a1m_1_protein.pdb',
 '1a1e_1_protein.pdb',
 '1a1o_1_protein.pdb',
 '1a1c_1_protein.pdb',
 '1a1a_1_protein.pdb',
 '1a1n_1_protein.pdb',
 '1a2b_1_protein.pdb',
 '1a2c_1_protein.pdb',
 '1a3u_1_protein.pdb',
 '1a2u_1_protein.pdb',
 '1a3k_1_protein.pdb',
 '1a2t_1_protein.pdb',
 '1a3t_1_protein.pdb',
 '1a3v_1_protein.pdb',
 '1a4g_1_protein.pdb',
 '1a4k_1_protein.pdb',
 '1a4h_1_protein.pdb',
 '1a4i_1_protein.pdb',
 '1a4m_2_protein.pdb',
 '1a4q_1_protein.pdb',
 '1a4m_4_protein.pdb',
 '1a4m_1_protein.pdb',
 '1a4m_3_protein.pdb',
 '1a4r_1_protein.pdb',
 '1a4k_2_protein.pdb',
 '1a4w_1_protein.pdb',
 '1a5b_1_protein.pdb',
 '1a5s_1_protein.pdb',
 '1a5u_1_protein.pdb',
 '1a5u_2_protein.pdb',
 '1a5z_1_protein.pdb',
 '1a5w_1_protein.pdb',
 '1a5x_1_protein.pdb',
 '1a5v_1_protein.pdb',
 '1a6v_1_protein.pdb']

### Creating FASTA files from PDB
We first convert the sequence information in PDB files to FASTA files, which the ESM library handles. The FASTA files should contain per-chain information for each protein.

In [203]:
from Bio import SeqIO
import time

for file in files:
  filepath = os.path.join(data_dir, file)
  for record in SeqIO.parse(filepath, "pdb-atom"):
    chain = record.annotations['chain']
    id = os.path.basename(file).split('.')[0]
    record.id = id + ':' + chain
    record.description = ''

    sequence_lengths[record.id] = len(record.seq)
    fasta_filename = os.path.join('fasta', id + '_' + chain + '.fasta')

    print(fasta_filename)
    with open(fasta_filename, "w") as output_handle:
      print('>' + record.id, file=output_handle)
      print(record.seq, file=output_handle)
      output_handle.flush()



/usr/local/lib/python3.10/dist-packages/Bio/PDB/PDBParser.py:395: PDBConstructionWarning: Ignoring unrecognized record 'TER' at line 2255
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/Bio/PDB/PDBParser.py:395: PDBConstructionWarning: Ignoring unrecognized record 'TER' at line 4528
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/Bio/SeqIO/PdbIO.py:322: BiopythonParserWarning: 'HEADER' line not found; can't determine PDB ID.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/Bio/PDB/PDBParser.py:395: PDBConstructionWarning: Ignoring unrecognized record 'TER' at line 2694
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/Bio/PDB/PDBParser.py:395: PDBConstructionWarning: Ignoring unrecognized record 'TER' at line 5387
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/Bio/SeqIO/PdbIO.py:322: BiopythonParserWarning: 'HEADER' line not found; can't determine PDB ID.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/Bio/PDB/PDBParser.py:395: PDBCon

fasta/1a0g_1_protein_A.fasta
fasta/1a0g_1_protein_B.fasta
fasta/1a05_1_protein_A.fasta
fasta/1a05_1_protein_B.fasta
fasta/1a0f_1_protein_A.fasta
fasta/1a0f_1_protein_B.fasta


/usr/local/lib/python3.10/dist-packages/Bio/PDB/PDBParser.py:395: PDBConstructionWarning: Ignoring unrecognized record 'TER' at line 4984
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/Bio/PDB/PDBParser.py:395: PDBConstructionWarning: Ignoring unrecognized record 'TER' at line 6645
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/Bio/SeqIO/PdbIO.py:322: BiopythonParserWarning: 'HEADER' line not found; can't determine PDB ID.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/Bio/PDB/PDBParser.py:395: PDBConstructionWarning: Ignoring unrecognized record 'TER' at line 1645
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/Bio/PDB/PDBParser.py:395: PDBConstructionWarning: Ignoring unrecognized record 'TER' at line 3186
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/Bio/SeqIO/PdbIO.py:322: BiopythonParserWarning: 'HEADER' line not found; can't determine PDB ID.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/Bio/PDB/PDBParser.py:395: PDBCon

fasta/1a0j_1_protein_A.fasta
fasta/1a0j_1_protein_B.fasta
fasta/1a0j_1_protein_C.fasta
fasta/1a0j_1_protein_D.fasta
fasta/1a0q_1_protein_H.fasta
fasta/1a0q_1_protein_L.fasta
fasta/1a0t_1_protein_P.fasta
fasta/1a0t_1_protein_Q.fasta
fasta/1a0t_1_protein_R.fasta
fasta/1a1b_1_protein_A.fasta
fasta/1a1b_1_protein_B.fasta


/usr/local/lib/python3.10/dist-packages/Bio/PDB/PDBParser.py:395: PDBConstructionWarning: Ignoring unrecognized record 'TER' at line 1022
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/Bio/PDB/PDBParser.py:395: PDBConstructionWarning: Ignoring unrecognized record 'TER' at line 2021
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/Bio/SeqIO/PdbIO.py:322: BiopythonParserWarning: 'HEADER' line not found; can't determine PDB ID.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/Bio/PDB/PDBParser.py:395: PDBConstructionWarning: Ignoring unrecognized record 'TER' at line 2281
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/Bio/PDB/PDBParser.py:395: PDBConstructionWarning: Ignoring unrecognized record 'TER' at line 3111
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/Bio/SeqIO/PdbIO.py:322: BiopythonParserWarning: 'HEADER' line not found; can't determine PDB ID.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/Bio/PDB/PDBParser.py:395: PDBCon

...

Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.10/dist-packages/Bio/PDB/PDBParser.py:416: PDBConstructionWarning: PDBConstructionException: Atom N defined twice in residue <Residue GLY het=  resseq=56 icode= > at line 1139.
Exception ignored.
Some atoms or residues may be missing in the data structure.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/Bio/PDB/PDBParser.py:416: PDBConstructionWarning: PDBConstructionException: Atom CA defined twice in residue <Residue GLY het=  resseq=56 icode= > at line 1140.
Exception ignored.
Some atoms or residues may be missing in the data structure.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/Bio/PDB/PDBParser.py:416: PDBConstructionWarning: PDBConstructionException: Atom C defined twice in residue <Residue GLY het=  resseq=56 icode= > at line 1141.
Exception ignored.
Some atoms or residues may be missing in the data structure.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/Bio/PDB/PDBParser

fasta/1a5w_1_protein_A.fasta


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



fasta/1a5v_1_protein_A.fasta
fasta/1a6v_1_protein_H.fasta
fasta/1a6v_1_protein_L.fasta


/usr/local/lib/python3.10/dist-packages/Bio/PDB/PDBParser.py:416: PDBConstructionWarning: PDBConstructionException: Atom CD1 defined twice in residue <Residue LEU het=  resseq=189 icode= > at line 2168.
Exception ignored.
Some atoms or residues may be missing in the data structure.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/Bio/PDB/PDBParser.py:416: PDBConstructionWarning: PDBConstructionException: Atom CD2 defined twice in residue <Residue LEU het=  resseq=189 icode= > at line 2169.
Exception ignored.
Some atoms or residues may be missing in the data structure.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/Bio/PDB/StructureBuilder.py:127: PDBConstructionWarning: WARNING: Residue (' ', 190, ' ') redefined at line 2170.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/Bio/PDB/StructureBuilder.py:149: PDBConstructionWarning: WARNING: Residue (' ', 190, ' ','ALA') already defined with the same name at line  2170.
  warnings.warn(
/usr/local/lib/python3.10/

### Generating ESM embeddings from FASTA
We use the FASTA files to generate ESM embeddings according to the ESM repo, which contains a python script to run the model and extract embeddings at certain layers.

In [ ]:
%%bash
export PATH="$HOME/.local/bin:$PATH"
git clone https://github.com/facebookresearch/esm.git


In [207]:
%%bash

for filename in fasta/*; do
  python esm/scripts/extract.py esm2_t33_650M_UR50D $filename \
    embeddings --repr_layers 0 32 33 --include mean per_tok
done


Read fasta/1a05_1_protein_A.fasta with 1 sequences
Processing 1 of 1 batches (1 sequences)
Read fasta/1a05_1_protein_B.fasta with 1 sequences
Processing 1 of 1 batches (1 sequences)
Read fasta/1a0f_1_protein_A.fasta with 1 sequences
Processing 1 of 1 batches (1 sequences)
Read fasta/1a0f_1_protein_B.fasta with 1 sequences
Processing 1 of 1 batches (1 sequences)
Read fasta/1a0g_1_protein_A.fasta with 1 sequences
Processing 1 of 1 batches (1 sequences)
Read fasta/1a0g_1_protein_B.fasta with 1 sequences
Processing 1 of 1 batches (1 sequences)
Read fasta/1a0j_1_protein_A.fasta with 1 sequences
Processing 1 of 1 batches (1 sequences)
Read fasta/1a0j_1_protein_B.fasta with 1 sequences
Processing 1 of 1 batches (1 sequences)
Read fasta/1a0j_1_protein_C.fasta with 1 sequences
Processing 1 of 1 batches (1 sequences)
Read fasta/1a0j_1_protein_D.fasta with 1 sequences
Processing 1 of 1 batches (1 sequences)
Read fasta/1a0q_1_protein_H.fasta with 1 sequences
Processing 1 of 1 batches (1 sequences)